## Day4-02: Agent Evaluation 代理评估

5 天 AI Agents 强化课程的第四天第二部分，我们将学习如何系统性地评估 Agent 的表现。

你将学会：

- 什么是 Agent 评估以及为什么它很重要
- 如何在 ADK Web UI 中创建和运行评估测试
- 理解评估指标：响应匹配分数和工具轨迹分数
- 如何使用 `adk eval` CLI 进行自动化回归测试
- 用户模拟 (User Simulation) 的概念

## 精简总结：Agent 评估

### 核心概念

- **问题**：传统测试 ≠ Agent 评估（Agent 是非确定性的）
- **解决方案**：系统性评估整个决策过程，包括最终响应和工具调用轨迹

```
                可观测性 + Agent 评估
                (被动响应)   (主动预防)
```

### 为什么 Agent 需要专门的评估？

| 传统软件 | AI Agent |
|---------|----------|
| 确定性输出 | 非确定性输出 |
| 可预测失败 | 失败方式不可预测 |
| 固定输入输出 | 用户输入模糊多变 |

### 评估指标

| 指标 | 含义 | 分数含义 |
|------|------|----------|
| `response_match_score` | 响应文本相似度 | 1.0 = 完全匹配，0.0 = 完全不同 |
| `tool_trajectory_avg_score` | 工具使用正确性 | 1.0 = 完美工具使用，0.0 = 错误工具/参数 |

### 评估方式选择

| 方式 | 适用场景 |
|------|----------|
| ADK Web UI | 交互式测试创建和分析 |
| `adk eval` CLI | 自动化批量回归测试 |
| pytest | 程序化测试集成到 CI/CD |

本次课程直播回放请到 youtube 观看: [youtube](https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE)

---
## Section 1: 设置环境

### 1.1 配置你的 Gemini API 密钥

In [ ]:
import os
from pathlib import Path

# 读取项目根目录的 .env 文件
env_file = Path.cwd().parent / '.env'

if env_file.exists():
    for line in env_file.read_text().splitlines():
        if line.startswith('GOOGLE_API_KEY='):
            os.environ["GOOGLE_API_KEY"] = line.split('=', 1)[1].strip()
            print("✅ Gemini API key 配置完成。")
            break
else:
    print(f"请在以下路径创建 .env 文件: {env_file}")

---
## Section 2: 创建家庭自动化 Agent

让我们创建一个将成为评估故事核心的 Agent。这个家庭自动化 Agent 在基本测试中看起来完美，但有我们将通过全面评估发现的隐藏缺陷。

运行 `adk create` CLI 命令来设置项目脚手架。

In [ ]:
!adk create home_automation_agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

运行下面的 cell 创建家庭自动化 Agent。

这个 Agent 使用单个 `set_device_status` 工具来控制智能家居设备。设备状态只能是 ON 或 OFF。**Agent 的指令故意过于自信** - 它声称可以控制 "所有智能设备" 和 "用户提到的任何设备" - 为我们将要发现的评估问题埋下伏笔。

In [ ]:
%%writefile home_automation_agent/agent.py

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini

from google.genai import types

# 配置模型重试错误处理
retry_config = types.HttpRetryOptions(
    attempts=5,  # 最大重试次数
    exp_base=7,  # 延迟乘数
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # 遇到这些 HTTP 错误时重试
)

def set_device_status(location: str, device_id: str, status: str) -> dict:
    """设置智能家居设备的状态。

    Args:
        location: 设备所在的房间。
        device_id: 设备的唯一标识符。
        status: 期望的状态，'ON' 或 'OFF'。

    Returns:
        确认操作的字典。
    """
    print(f"工具调用: 将 {location} 的 {device_id} 设置为 {status}")
    return {
        "success": True,
        "message": f"成功将 {location} 的 {device_id} 设置为 {status.lower()}。"
    }

# 这个 Agent 有故意设计的缺陷，我们将通过评估发现它们！
root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="home_automation_agent",
    description="一个控制家庭智能设备的 Agent。",
    instruction="""你是一个家庭自动化助手。你控制房子里的所有智能设备。
    
    你可以访问灯光、安防系统、烤箱、壁炉以及用户提到的任何其他设备。
    始终尝试帮助并控制用户要求的任何设备。
    
    当用户询问设备功能时，告诉他们你能控制的所有令人惊叹的功能。""",
    tools=[set_device_status],
)

---
## Section 3: 使用 ADK Web UI 进行交互式评估

### 3.1 启动 ADK Web UI

在终端中运行以下命令启动 ADK Web UI：

```bash
adk web home_automation_agent
```

然后在浏览器中打开 http://localhost:8000

In [ ]:
# 或者直接在 notebook 中运行（会持续运行直到手动停止）
# !adk web home_automation_agent

### 3.2 创建你的第一个 "完美" 测试用例

**👉 操作：在 ADK Web UI 中**

1. 点击上面的 URL 打开 ADK Web UI
2. 从下拉菜单中选择 "home_automation_agent"
3. **进行正常对话：** 输入 `Turn on the desk lamp in the office`（打开办公室的台灯）
4. **Agent 正确响应** - 控制设备并确认操作

**👉 操作：将此保存为你的第一个评估用例**

1. 导航到右侧面板的 **Eval** 标签页
2. 点击 **Create Evaluation set** 并命名为 `home_automation_tests`
3. 在 `home_automation_tests` 集合中，点击 ">" 箭头并点击 **Add current session**
4. 给它命名为 `basic_device_control`

**✅ 成功！** 你刚刚将第一个交互保存为评估用例。

![创建测试用例](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/eval-create-testcase.gif)

### 3.3 运行评估

**👉 操作：运行你的第一个评估**

现在，让我们运行测试用例看看 Agent 能否复制之前的成功。

1. 在 Eval 标签页中，确保你的新测试用例被勾选
2. 点击 Run Evaluation 按钮
3. EVALUATION METRIC 对话框会出现。现在保持默认值并点击 Start
4. 评估将运行，你应该在 Evaluation History 中看到绿色的 Pass 结果。这确认 Agent 的行为与保存的会话匹配

‼️ **理解评估指标**

当你运行评估时，你会看到两个关键分数：

* **Response Match Score（响应匹配分数）：** 衡量 Agent 的实际响应与预期响应的相似程度。使用文本相似度算法比较内容。分数 1.0 = 完全匹配，0.0 = 完全不同。

* **Tool Trajectory Score（工具轨迹分数）：** 衡量 Agent 是否使用了正确的工具和正确的参数。检查工具调用序列是否符合预期行为。分数 1.0 = 完美工具使用，0.0 = 错误工具或参数。

**👉 操作：分析失败**

让我们故意破坏测试来看看失败是什么样的。

1. 在评估用例列表中，点击测试用例旁边的编辑（铅笔）图标
2. 在 "Final Response" 文本框中，将预期文本改成错误的，如：`The desk lamp is off`（台灯已关闭）
3. 保存更改并重新运行评估
4. 这次，结果将是红色的 Fail。将鼠标悬停在 "Fail" 标签上。会出现一个工具提示，显示实际输出与预期输出的并排比较，精确地显示测试为什么失败（最终响应不匹配）

这种即时、详细的反馈对于调试非常有价值。

![评估](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/eval-run-test.gif)

### 3.4 （可选）创建有挑战性的测试用例

现在创建更多测试用例来暴露隐藏的问题：

**在单独的对话中创建这些场景：**

1. **模糊的命令：** `"Turn on the lights in the bedroom"`（打开卧室的灯）
   - 保存为新测试用例：`ambiguous_device_reference`
   - 运行评估 - 它可能通过，但 Agent 可能会困惑

2. **无效的位置：** `"Please turn off the TV in the garage"`（请关闭车库的电视）
   - 保存为新测试用例：`invalid_location_test`
   - 运行评估 - Agent 可能会尝试控制不存在的设备

3. **复杂的命令：** `"Turn off all lights and turn on security system"`（关闭所有灯并打开安防系统）
   - 保存为新测试用例：`complex_multi_device_command`
   - 运行评估 - Agent 可能会尝试超出其能力范围的操作

**你将发现的问题：**
即使测试 "通过"，你也可以看到 Agent：
- 对不存在的设备做出假设
- 给出听起来有帮助但不准确的响应
- 尝试控制它不应该访问的设备

## 🤔 我还缺少什么？

❌ **Web UI 的局限性：** 到目前为止，我们已经看到如何在 ADK Web UI 中创建和评估测试用例。Web UI 非常适合交互式测试创建，但一次测试一个对话无法扩展。

❓ **问题：** 我如何主动检测 Agent 性能的回归？

让我们在下一节中回答这个问题！

---

## ‼️ **停止 ADK Web UI** 🛑

**为了运行本 notebook 后续部分的代码，** 请停止你启动 `adk web` 的运行中的 cell。

否则，只要 ADK Web UI 在运行，那个运行中的 cell 就会阻止其他 cell 运行。

---
## Section 4: 系统性评估

回归测试是重新运行现有测试的实践，以确保新更改没有破坏之前正常工作的功能。

ADK 提供两种方法进行自动回归和批量测试：使用 [pytest](https://google.github.io/adk-docs/evaluate/#2-pytest-run-tests-programmatically) 和 [adk eval](https://google.github.io/adk-docs/evaluate/#3-adk-eval-run-evaluations-via-the-cli) CLI 命令。在本节中，我们将使用 CLI 命令。有关 `pytest` 方法的更多信息，请参阅本 notebook 末尾资源部分的链接。

下图显示了评估的整体流程。**在高层次上，评估有四个步骤：**

1) **创建评估配置** - 定义指标或你想要衡量的内容
2) **创建测试用例** - 用于比较的样本测试用例
3) **使用测试查询运行 Agent**
4) **比较结果**

![评估流程](https://storage.googleapis.com/github-repo/kaggle-5days-ai/day4/evaluate_agent.png)

### 4.1 创建评估配置

这个可选文件让我们定义通过/失败的阈值。在根目录中创建 `test_config.json`。

In [ ]:
import json

# 创建带有基本标准的评估配置
eval_config = {
    "criteria": {
        "tool_trajectory_avg_score": 1.0,  # 要求完美的工具使用
        "response_match_score": 0.8,  # 80% 文本相似度阈值
    }
}

with open("home_automation_agent/test_config.json", "w") as f:
    json.dump(eval_config, f, indent=2)

print("✅ 评估配置创建完成！")
print("\n📊 评估标准:")
print("• tool_trajectory_avg_score: 1.0 - 要求精确的工具使用匹配")
print("• response_match_score: 0.8 - 要求 80% 文本相似度")
print("\n🎯 这个评估将捕获:")
print("✅ 错误的工具使用（错误的设备、位置或状态）")
print("✅ 糟糕的响应质量和沟通")
print("✅ 偏离预期行为模式")

### 4.2 创建测试用例

这个文件（`integration.evalset.json`）将包含多个测试用例（会话）。

这个评估集可以合成创建，也可以从 ADK Web UI 中的对话会话创建。

**提示：** 要从 ADK Web UI 持久化对话，只需在 UI 中创建一个 evalset 并将当前会话添加到其中。该会话中的所有对话将自动转换为 evalset 并下载到本地。

In [ ]:
# 创建揭示工具使用和响应质量问题的评估测试用例
test_cases = {
    "eval_set_id": "home_automation_integration_suite",
    "eval_cases": [
        {
            "eval_id": "living_room_light_on",
            "conversation": [
                {
                    "user_content": {
                        "parts": [
                            {"text": "请打开客厅的落地灯"}
                        ]
                    },
                    "final_response": {
                        "parts": [
                            {
                                "text": "成功将客厅的落地灯设置为 on。"
                            }
                        ]
                    },
                    "intermediate_data": {
                        "tool_uses": [
                            {
                                "name": "set_device_status",
                                "args": {
                                    "location": "客厅",
                                    "device_id": "落地灯",
                                    "status": "ON",
                                },
                            }
                        ]
                    },
                }
            ],
        },
        {
            "eval_id": "kitchen_light_on",
            "conversation": [
                {
                    "user_content": {
                        "parts": [{"text": "打开厨房的主灯。"}]
                    },
                    "final_response": {
                        "parts": [
                            {
                                "text": "成功将厨房的主灯设置为 on。"
                            }
                        ]
                    },
                    "intermediate_data": {
                        "tool_uses": [
                            {
                                "name": "set_device_status",
                                "args": {
                                    "location": "厨房",
                                    "device_id": "主灯",
                                    "status": "ON",
                                },
                            }
                        ]
                    },
                }
            ],
        },
    ],
}

让我们将测试用例写入 Agent 根目录中的 `integration.evalset.json`。

In [ ]:
import json

with open("home_automation_agent/integration.evalset.json", "w") as f:
    json.dump(test_cases, f, indent=2, ensure_ascii=False)

print("✅ 评估测试用例创建完成")
print("\n🧪 测试场景:")
for case in test_cases["eval_cases"]:
    user_msg = case["conversation"][0]["user_content"]["parts"][0]["text"]
    print(f"• {case['eval_id']}: {user_msg}")

print("\n📊 预期结果:")
print("• basic_device_control: 应该通过两个标准")
print("• wrong_tool_usage_test: 如果 agent 使用错误参数可能会失败 tool_trajectory")
print("• poor_response_quality_test: 如果响应差异太大可能会失败 response_match")

### 4.3 运行 CLI 评估

执行 `adk eval` 命令，指向你的 agent 目录、evalset 和配置文件。

In [ ]:
print("🚀 运行此命令执行评估:")
!adk eval home_automation_agent home_automation_agent/integration.evalset.json --config_file_path=home_automation_agent/test_config.json --print_detailed_results

### 4.4 分析评估结果示例

该命令将运行所有测试用例并打印摘要。`--print_detailed_results` 标志提供每个测试的逐轮细分，显示分数和任何失败的差异。

In [ ]:
# 分析评估结果 - 数据科学方法
print("📊 理解评估结果:")
print()
print("🔍 示例分析:")
print()
print("测试用例: living_room_light_on")
print("  ❌ response_match_score: 0.45/0.80")
print("  ✅ tool_trajectory_avg_score: 1.0/1.0")
print()
print("📈 这告诉我们什么:")
print("• 工具使用: 完美 - Agent 使用了正确的工具和正确的参数")
print("• 响应质量: 差 - 响应文本与预期差异太大")
print("• 根本原因: Agent 的沟通风格问题，而非功能问题")
print()
print("🎯 可操作的洞察:")
print("1. 技术能力正常（工具使用完美）")
print("2. 沟通需要改进（响应质量失败）")
print("3. 修复: 更新 agent 指令以获得更清晰的语言或受约束的响应。")
print()

---
## Section 5: 用户模拟（可选）

虽然**传统评估方法依赖固定的测试用例**，但现实世界的对话是动态且不可预测的。这就是用户模拟发挥作用的地方。

用户模拟是 ADK 中一个强大的功能，解决了静态评估的局限性。它不使用预定义的固定用户提示，而是使用生成式 AI 模型（如 Gemini）在评估过程中**动态生成用户提示**。

### ❓ 它是如何工作的

* 你定义一个 `ConversationScenario`，概述用户的整体对话目标和一个 `conversation_plan` 来指导对话。
* 然后，一个大型语言模型（LLM）充当模拟用户，使用这个计划和正在进行的对话历史来生成现实且多样的提示。
* 这允许对你的 agent 处理意外转折、维护上下文以及在更自然、不可预测的对话流中实现复杂目标的能力进行更全面的测试。

用户模拟帮助你发现静态测试用例经常遗漏的边缘情况，并提高你的 agent 的健壮性。

### 👉 练习

现在你了解了用户模拟在动态 agent 评估中的强大功能，这里有一个练习供你应用：

将**用户模拟**功能应用到你的 agent。为特定目标定义一个带有 `conversation_plan` 的 `ConversationScenario`，并将其集成到你的 agent 评估中。

**⭐ 参考这个[文档](https://google.github.io/adk-docs/evaluate/user-sim/)了解如何做。**

## 🏆 恭喜！

### 你学到了

- ✅ 在 ADK Web UI 中进行交互式测试创建和分析
- ✅ 工具轨迹和响应指标
- ✅ 使用 `adk eval` CLI 命令进行自动化回归测试
- ✅ 如何分析评估结果并据此修复 agent

### 📚 资源

* [ADK 评估概述](https://google.github.io/adk-docs/evaluate/)
* 不同的[评估标准](https://google.github.io/adk-docs/evaluate/criteria/)
* [基于 Pytest 的评估](https://google.github.io/adk-docs/evaluate/#2-pytest-run-tests-programmatically)

### 高级评估

对于生产部署，ADK 支持[高级标准](https://docs.cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval)如 `safety_v1` 和 `hallucinations_v1`（需要 Google Cloud 凭据）。

### 🎯 下一步

准备好迎接下一个挑战了吗？请继续关注最后的 Day 5 notebooks，我们将把所有内容汇总！😎

我们将学习如何**将 Agent 部署到生产环境**以及使用 **Agent2Agent 协议**扩展它们。